In [1]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection, metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection  import GridSearchCV
import time

#load data
config = json.load(open('settings.json'))
train = pd.read_csv(config['train_modified'])
test = pd.read_csv(config['test_modified'])
num_classes = 8

from sklearn.model_selection import train_test_split
train_part, test_part = train_test_split(train, test_size=0.2)

target='Response'
IDcol = 'Id'

predictors = [x for x in train_part.columns if x not in [target, IDcol]]

X=train_part[predictors]
y=train_part[target]

In [2]:
X1=train_part
X1['Response'] -=1 #reduce to fit [0,classes)
y1=test_part
y1['Response'] -= 1

C:\Users\dli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\dli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
y1["Response"].value_counts()

7    3938
5    2246
6    1568
1    1283
0    1229
4    1109
3     289
2     215
Name: Response, dtype: int64

In [4]:
param = {'max_depth' : 4, 
         'eta' : 0.01, 
         'silent' : 1, 
         'min_child_weight' : 10, 
         'subsample' : 0.5,
         'early_stopping_rounds' : 100,
         'objective' : 'multi:softprob',
         'num_class' : 8,
         'colsample_bytree' : 0.3,
         'seed' : 0}
num_rounds=7000
dtrain=xgb.DMatrix(X1[predictors],X1[target],missing=float('nan'))

In [5]:
%time bst = xgb.train(param, dtrain, num_rounds)

Wall time: 50min 59s


In [7]:
dtest=xgb.DMatrix(y1[predictors],y1[target],missing=float('nan'))
%time prob = bst.predict(dtest)

Wall time: 16 s


In [35]:
y=np.argmax(prob, axis=1)
print ('Accuracy:%.4g'%metrics.accuracy_score(y1[target],y))

Accuracy:0.5893


In [13]:
prob

array([[1.1095951e-02, 6.0166968e-03, 1.2473860e-04, ..., 4.4593506e-02,
        1.8648732e-01, 7.4519145e-01],
       [1.6468041e-02, 3.2978080e-02, 2.1740847e-04, ..., 6.7996308e-02,
        8.0372199e-02, 7.7966231e-01],
       [1.5194499e-02, 1.6266905e-02, 8.1960665e-04, ..., 2.3765322e-02,
        2.9402457e-02, 9.0318614e-01],
       ...,
       [2.4412753e-01, 4.6766394e-01, 8.0492767e-04, ..., 1.4751874e-01,
        1.5099860e-04, 8.5396176e-05],
       [2.0459896e-01, 2.1885158e-01, 6.0502584e-03, ..., 5.3171140e-01,
        4.4854917e-03, 1.2535411e-02],
       [1.1551266e-02, 1.0190281e-02, 3.2489284e-04, ..., 5.4286085e-02,
        1.7385477e-02, 8.9858842e-01]], dtype=float32)

In [40]:
y0 = test
y0[target] -= 1
dtest=xgb.DMatrix(y0[predictors],y0[target],missing=float('nan'))
%time prob2 = bst.predict(dtest)
y=np.argmax(prob2, axis=1)
y += 1
result = pd.DataFrame({"Id": y0['Id'].values, "Response": y})
result.to_csv('../src/submissions/submission_xg2.csv', index=False)

Wall time: 28.7 s
